## View example downloaded EMIT agricultural scenes

We will use the image granules that you orthorectified in the previous step **"1_Orthorectify_images.ipynb"**

### Step 1. Setup notebook

Import packages

In [12]:
import os, sys, fnmatch
import warnings
from osgeo import gdal
import numpy as np
import math
import rasterio as rio
import xarray as xr
import holoviews as hv
import hvplot.xarray
import pandas as pd
import hvplot.pandas
import geopandas as gpd
import rioxarray as rxr
import numpy as np
import folium
from branca.element import Figure

# Get custom functions -- MAY NOT NEED THIS STEP
#sys.path.append(os.path.join(os.path.expanduser("~"),"HYR-SENSE","tools","functions"))
#from helper_functions import *

# Setup paths
datadir = os.path.join(os.path.expanduser('~'),'HYR-SENSE/data/Agriculture/')
outdir = os.path.join(os.path.expanduser('~'),'HYR-SENSE/data/Agriculture/output')
workflow = "Agriculture"
platform = "emit"
source_file_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data",workflow,platform)

# This will ignore some warnings caused by holoviews
warnings.simplefilter('ignore')

### Step 2. Find all downloaded and orthorectified agricultural images

In [3]:
# Define workflow which selects the appropriate image data folder
print("*** EMIT data folder: " + source_file_path)
print("")
print("*** GIS data folder: " + datadir)
print("")

*** EMIT data folder: /home/jovyan/HYR-SENSE/data/Agriculture/emit

*** GIS data folder: /home/jovyan/HYR-SENSE/data/Agriculture/



In [4]:
### List all of the available data located in the EMIT data folder
granules = fnmatch.filter(os.listdir(source_file_path), '*ortho.nc')
print("*** EMIT Data Granules ***")
granules

*** EMIT Data Granules ***


['EMIT_L2A_RFL_001_20230729T205618_2321014_018_ortho.nc',
 'EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc',
 'EMIT_L2A_RFL_001_20230729T205642_2321014_020_ortho.nc',
 'EMIT_L2A_RFL_001_20230729T205653_2321014_021_ortho.nc']

### Step 3. Select and load a previously orthorectified EMIT image

First let's remind ourselves where we currently EMIT data ready to be analyzed.  We can do this by re-plotting the EMIT bounding boxes that we saved in the data search and download step

In [18]:
### Load the EMIT bounding boxes and plot on the map
gdf_all = gpd.read_file(os.path.join(datadir,'emit_granule_footprints.gpkg'))
# Filter to the Yuma, CO granule
gdf_yuma = gdf_all[gdf_all['meta.native-id'] == 'EMIT_L2A_RFL_001_20230729T205630_2321014_019']
gdf_yuma = gdf_yuma.to_crs(5070)
gdf_all = gdf_all.to_crs(5070)

In [19]:
print("*** EMIT granule bounding boxes ***")
gdf_all

*** EMIT granule bounding boxes ***


,meta.native-id,start_datetime,geometry
0,EMIT_L2A_RFL_001_20230729T205618_2321014_018,2023-07-29T20:56:18Z,"POLYGON ((-613275.227 1950433.936, -698639.073..."
1,EMIT_L2A_RFL_001_20230729T205630_2321014_019,2023-07-29T20:56:30Z,"POLYGON ((-545459.565 1996275.527, -631184.395..."
2,EMIT_L2A_RFL_001_20230729T205642_2321014_020,2023-07-29T20:56:42Z,"POLYGON ((-477688.718 2042003.979, -563588.787..."
3,EMIT_L2A_RFL_001_20230729T205653_2321014_021,2023-07-29T20:56:53Z,"POLYGON ((-409655.435 2087509.537, -495981.960..."


In [36]:
# Create the interactive map using folium
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

gdf_all.explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)

map1.fit_bounds(map1.get_bounds(), padding=(30, 30))
display(fig)

For now, let's select a single scene ID to explore the data more closely. We will choose the scene that covers Yuma Colorado, shown in blue below

In [38]:
### Show just the Yuma CO scene
gdf_yuma.explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.91, width=5),
    name="EMIT",
    m=map1,
    legend=False
)
map1.fit_bounds(map1.get_bounds(), padding=(30, 30))
display(fig)

In [39]:
### Pick an image granule to explore - we will start with EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc
img_file = 'EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc'

In [40]:
# # Load the selected image to memory
img_file_dat = os.path.join(source_file_path,img_file)
ds_geo = xr.open_dataset(img_file_dat)
ds_geo

<xarray.Dataset>
Dimensions:           (latitude: 1874, longitude: 2534, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 381.0 388.4 ... 2.486e+03 2.493e+03
    fwhm              (wavelengths) float32 ...
    good_wavelengths  (wavelengths) float32 ...
  * latitude          (latitude) float64 40.69 40.69 40.69 ... 39.67 39.67 39.67
  * longitude         (longitude) float64 -103.5 -103.5 -103.5 ... -102.1 -102.1
    elev              (latitude, longitude) float32 ...
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 ...
    spatial_ref       int64 ...
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-1.03490233e+02  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20230729T205630_23210...
    Orthorectified:                    True

### Step 4. Quickly display the selected orthorectified image

Here we will view the selected orthorectified image that contains Yuma Colorado, shown below with the yellow do

In [ ]:
refl850 = ds_geo.sel(wavelengths=850, method='nearest')
yuma_df = [[40.1222,-102.7252]]
yuma_df = pd.DataFrame(yuma_df, columns=['Latitude', 'Longitude'])

img_plot = ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl850.wavelengths.values:.3f} {refl850.wavelengths.units} (Orthorectified)")
pt_plot = yuma_df.hvplot.points("Longitude", "Latitude", geo=True, color="yellow", alpha=0.9, s=250, global_extent=False)
img_plot * pt_plot

We can see that the orthorectification step placed the data on a geogrpahic grid that matches pretty well with ESRI tiles. Now that we have a better idea of what the target area looks like, we can also plot the spectra using the georeferenced data. 

Similarly, we can review a wavelength in the visible spectrum.  Let's use a wavelength found within the green wavelengths 

In [ ]:
refl550 = ds_geo.sel(wavelengths=550, method='nearest')
ds_geo.sel(wavelengths=550, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl550.wavelengths.values:.3f} {refl550.wavelengths.units} (Orthorectified)")

In [ ]:
refl650 = ds_geo.sel(wavelengths=650, method='nearest')
ds_geo.sel(wavelengths=650, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl650.wavelengths.values:.3f} {refl650.wavelengths.units} (Orthorectified)")

We can also display all three bands side-by-side to look a the differences in reflectance at different wavelengths from the visible to the near-infrared

In [ ]:
refl550.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 550") + \
refl650.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 650") + \
refl850.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 850")

### Step 5. Plot example spectra

Now let's plot some example spectra found in the image. Before we do this, we should filter out the water absorption bands like we did earlier. By limiting the third dimension of the array to good_wavelengths.

In [ ]:
ds_geo['reflectance'].data[:,:,ds_geo['good_wavelengths'].data==0] = np.nan

In [ ]:

point = ds_geo.sel(longitude=-102.694,latitude=40.347,method='nearest')
point.hvplot.line(y='reflectance',x='wavelengths', color='black', frame_height=400, frame_width=600).opts(
    title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')

#### Look at spectra across major crop types

DRAFT.

Load data from the Cropland Data Layer (CDL) which provides a raster image of different crop types.

Clip to our EMIT granule region.

Identify the top 10 most common classes.

Create a random point sample of the top three most common crops.

In [ ]:
# Load the Cropland Data Layer (CDL) for our region
# Crop to our Yuma, CO footprint
import geopandas as gpd
import rioxarray as rxr
import numpy as np

datadir = '/home/jovyan/HYR-SENSE/data/Agriculture/'

gdf = gpd.read_file(os.path.join(datadir,'emit_granule_footprints.gpkg'))
# Filter to the Yuma, CO granule
gdf = gdf[gdf['meta.native-id'] == 'EMIT_L2A_RFL_001_20230729T205630_2321014_019']
gdf = gdf.to_crs(5070)

# Load the CDL raster
cdl_path = os.path.join(datadir,'CDL_2023_CO_SouthPlatte_Republican.tif')
cdl = rxr.open_rasterio(cdl_path, mask=True, cache=False).squeeze()
cdl = cdl.rio.clip(gdf.geometry)
print(cdl)
unique_values = np.unique(cdl.values)
print(unique_values)

# Get the most common pixel values by counting the frequency
cdl_values = cdl.values.flatten()
cdl_values = cdl_values[cdl_values > 0]
counts = np.bincount(cdl_values)

# Get the indices of the ten most common pixel values
top_ten_indices = np.argsort(counts)[-10:][::-1]
top_ten_counts = counts[top_ten_indices]
print("Ten most common pixel values and their counts:", list(zip(top_ten_indices, top_ten_counts)))

# Convert the counts to a DataFrame
counts_df = pd.DataFrame({'Codes': top_ten_indices, 'Counts': top_ten_counts})
print("Counts DataFrame:")

# Load the crop type lookup table
lookup = pd.read_csv(os.path.join(datadir,'CDL_codes.csv'))

# Join the counts DataFrame with the lookup table
result_df = counts_df.merge(lookup, on='Codes', how='left')
print("Top ten most common crop types and their pixel values:")
print(result_df)

In [ ]:
from shapely.geometry import Point

major_crops = [24,1,29]
# Generate 10 random points within each of the three classes
points = []
for code in major_crops:
    # Find the indices of the pixels that belong to the current class
    indices = np.column_stack(np.where(cdl.values == code))
    for _ in range(10):
        # Randomly select an index
        rand = indices[np.random.choice(len(indices))]
        y, x = cdl.y[rand[0]].values, cdl.x[rand[1]].values
        point = Point(x, y)
        points.append((point, code))

# Create a GeoDataFrame with the sample points
gdf_samples = gpd.GeoDataFrame(points, columns=['geometry', 'Codes'], crs=cdl.rio.crs)

# Join with the lookup table to get the crop types
gdf_samples = gdf_samples.merge(lookup, left_on='Codes', right_on='Codes', how='left')
gdf_samples = gdf_samples.to_crs(4326)
gdf_samples['Latitude'] = gdf_samples.geometry.y
gdf_samples['Longitude'] = gdf_samples.geometry.x
print(gdf_samples)

### Plot average spectra for the three major crop types

Now that we have a random sample from the major crop types in our granule, we can plot the spectra at these points to examine any difference between the crop types.

NOTE: need to figure out why the points fall outside the EMIT granule?

In [ ]:
# Plot the EMIT data
df_samples = pd.DataFrame(gdf_samples.drop(columns='geometry'))

emit_plot = ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(
    cmap='greys',
    frame_height=600,
    frame_width=600,
    geo=True,
    crs='EPSG:4326'
).opts(title="Major Crop Type Samples")

# Plot the sample points
points_plot = df_samples.hvplot.points(
    x='Longitude',
    y='Latitude',
    by='Codes',
    color=hv.Cycle('Dark2'),
    geo=True,
    crs='EPSG:4326'
)

# Combine the plots
combined_plot = emit_plot * points_plot
combined_plot

In [ ]:
samples = df_samples.set_index(['Codes'])
xp = samples.to_xarray()
xp

In [ ]:
extracted = ds_geo.sel(latitude=xp.Latitude,longitude=xp.Longitude, method='nearest').to_dataframe()
extracted

NOTE: need to figure out the random points being outside the bounds. Also need to probably create an average for each crop type and plot that. Then this plot won't look so terrible ...

In [ ]:
extracted.hvplot(
    x='wavelengths',y='reflectance', by=['Codes'], 
    color=hv.Cycle('Dark2'), 
    frame_height=400, frame_width=600
).opts(title='Example Points - Reflectance', xlabel='Wavelengths (nm)',ylabel='Reflectance')

### Step 6. Experiment with band ratios or what are called Specrtal Vegetation Indices (SVIs)

To explore the utility of high spectral resolution data for calculating SVIs for Earth Science, we can demonstrate how to caluclate a commonly-used index: The Normalized Difference Vegetation Index (NDVI).  NDVI has been used for over 40 years to study changes on the Earth's surface, specifically related to vegetation, stress, and agriculture. For more information on NDVI, you can explore this article from NASA: [https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php)

In addition, here is a recent article describing the use of SVIs in Earth Science <br>
[https://www.nature.com/articles/s41597-023-02096-0](https://www.nature.com/articles/s41597-023-02096-0)

Calculate the Normalized Difference Vegetation Index (NDVI)

In [ ]:
# NDVI uses a combination of reflectance in the NIR and Red wavelengths.  For example, 850 and 655 nm
# First, let's calculate an NDVI image and take a look at the results
refl650 = ds_geo.sel(wavelengths=650, method='nearest')
refl850 = ds_geo.sel(wavelengths=850, method='nearest')
ndvi = (refl850-refl650)/(refl850+refl650)

In [ ]:
ndvi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

In [ ]:
### STILL A WIP!!!

### Boxplots of SVIs across crop types

### Boxplots of SVIs for irrigated vs. non-irrigated

In [ ]:
# Load the irrigated lands raster
irrigated = rxr.open_rasterio(os.path.join(datadir,'LANID_Irrigation_CO_SouthPlatte_Republican.tif'))
print(irrigated)

In [ ]:
# Create a data stack of CDL, irrigation, and EMIT

if not ds_geo.rio.crs:
    ds_geo = ds_geo.rio.write_crs("EPSG:4326")  # Set to the appropriate CRS if known

# Reproject and match our CDL and irrigation layers
cdl_repr = cdl.rio.reproject_match(ds_geo)
irrigated_repr = irrigated.rio.reproject_match(ds_geo)

ds_geo_da = ds_geo['emit_data'] if 'emit_data' in ds_geo.data_vars else ds_geo.to_array().squeeze()

# Stack the data layers
ds_stack = xr.Dataset({
    'emit': ds_geo_da,
    'cdl': cdl_repr,
    'irrigation': irrigated_repr
})

print(ds_stack)

In [ ]:
wavelengths = ds_stack['emit'].coords['wavelengths']
wavelengths